In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/child-mind-institute-detect-sleep-states/train_series.parquet
/kaggle/input/child-mind-institute-detect-sleep-states/sample_submission.csv
/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv
/kaggle/input/child-mind-institute-detect-sleep-states/test_series.parquet
/kaggle/input/zzzs-lightweight-training-dataset-target/Zzzs_train.parquet
/kaggle/input/zzzs-lightweight-training-dataset-target/Zzzs_train_multi.parquet


# Data

In [2]:
train=pd.read_parquet('/kaggle/input/zzzs-lightweight-training-dataset-target/Zzzs_train.parquet')

In [3]:
train.head(10)

,series_id,step,timestamp,anglez,enmo,awake
0,08db4255286f,0,2018-11-05T10:00:00-0400,-30.845301,0.0447,1
1,08db4255286f,1,2018-11-05T10:00:05-0400,-34.181801,0.0443,1
2,08db4255286f,2,2018-11-05T10:00:10-0400,-33.877102,0.0483,1
3,08db4255286f,3,2018-11-05T10:00:15-0400,-34.282101,0.0680,1
4,08db4255286f,4,2018-11-05T10:00:20-0400,-34.385799,0.0768,1
5,08db4255286f,5,2018-11-05T10:00:25-0400,-34.925598,0.0511,1
6,08db4255286f,6,2018-11-05T10:00:30-0400,-30.513399,0.1073,1
7,08db4255286f,7,2018-11-05T10:00:35-0400,-30.509399,0.0649,1
8,08db4255286f,8,2018-11-05T10:00:40-0400,-32.880600,0.0485,1
9,08db4255286f,9,2018-11-05T10:00:45-0400,-34.674999,0.0462,1


In [6]:
test=pd.read_parquet('/kaggle/input/child-mind-institute-detect-sleep-states/test_series.parquet')

In [7]:
test.head(10)

,series_id,step,timestamp,anglez,enmo
0,038441c925bb,0,2018-08-14T15:30:00-0400,2.6367,0.0217
1,038441c925bb,1,2018-08-14T15:30:05-0400,2.6368,0.0215
2,038441c925bb,2,2018-08-14T15:30:10-0400,2.6370,0.0216
3,038441c925bb,3,2018-08-14T15:30:15-0400,2.6368,0.0213
4,038441c925bb,4,2018-08-14T15:30:20-0400,2.6368,0.0215
5,038441c925bb,5,2018-08-14T15:30:25-0400,2.6367,0.0217
6,038441c925bb,6,2018-08-14T15:30:30-0400,2.6367,0.0217
7,038441c925bb,7,2018-08-14T15:30:35-0400,2.6367,0.0218
8,038441c925bb,8,2018-08-14T15:30:40-0400,2.7980,0.0223
9,038441c925bb,9,2018-08-14T15:30:45-0400,3.0847,0.0217


# Feature Engineering

In [19]:
def feature_programming(df,delta_t=[5,10]):
  
    #timestamp
  df['timestamp']=pd.to_datetime(df['timestamp'],utc=True)
  df['hour']=df['timestamp'].dt.hour
      #week
  df['week_name']=df['timestamp'].dt.day_name()
  df['week_name']=df['week_name'].replace({'Monday':1,'Tuesday':2,'Wednesday':3,'Thursday':4,'Friday':5,'Saturday':6,'Sunday':7})
  df['week_name']=df['week_name'].astype('int')

  #for 0th order：anglez and enmo
  df['anglez'].astype('float32')
  df['enmo'].astype('float32')

  #1)perform shift
  for i in delta_t:
    #sma as 0th basic feature
    df['anglez_roll_'+str(i)]=df.groupby(by='series_id',as_index=False)['anglez'].rolling(window=i).mean().fillna(method='bfill').drop('series_id',axis=1).astype('float32')
    df['enmo_roll_'+str(i)]=df.groupby(by='series_id',as_index=False)['enmo'].rolling(window=i).mean().fillna(method='bfill').drop('series_id',axis=1).astype('float32')
  #2)perform difference
    #first difference
    df['anglez_momentum_'+str(i)]=df['anglez']-df['anglez_roll_'+str(i)]
    df['enmo_momentum_'+str(i)]=df['enmo']-df['enmo_roll_'+str(i)]
  #for 1th order
  #1st basic feature
    df['ratio_ang_'+str(i)]=(df['anglez_momentum_'+str(i)]/df['anglez_roll_'+str(i)])
    df['ratio_enm_'+str(i)]=(df['enmo_momentum_'+str(i)]/df['enmo_roll_'+str(i)])
  return df


#denoise and smooth
def window(series,lookback_size):
  #denoise
  name_1='anglez_mean'+str(lookback_size)
  df[name_1]=df.groupby('series_id')['anglez'].rolling(window=lookback_size).mean().fillna(method='bfill').astype('float16')
  return df

def difference(df):
  #difference operator /
  #first difference
  df['anglez_momentum']=df.groupby('series_id')['anglez'].diff().fillna(method='bfill').astype('float16')
  #second difference
  df['anglez_acceleration']=df.groupby('series_id')['anglez_momentum'].diff().fillna(method='bfill').astype('float16')
  return df

In [20]:
train=feature_programming(train,delta_t=[5,10,15,20,25,30,40,50,60])

In [21]:
train.head(10)

,series_id,step,timestamp,anglez,enmo,awake,hour,week_name,anglez_roll_5,enmo_roll_5,...,anglez_momentum_50,enmo_momentum_50,ratio_ang_50,ratio_enm_50,anglez_roll_60,enmo_roll_60,anglez_momentum_60,enmo_momentum_60,ratio_ang_60,ratio_enm_60
0,08db4255286f,0,2018-11-05 14:00:00+00:00,-30.845301,0.0447,1,14,1,-33.514420,0.05642,...,-1.914207,-0.025428,0.066164,-0.362594,-28.294451,0.074093,-2.550850,-0.029393,0.090154,-0.396707
1,08db4255286f,1,2018-11-05 14:00:05+00:00,-34.181801,0.0443,1,14,1,-33.514420,0.05642,...,-5.250708,-0.025828,0.181490,-0.368298,-28.294451,0.074093,-5.887350,-0.029793,0.208074,-0.402105
2,08db4255286f,2,2018-11-05 14:00:10+00:00,-33.877102,0.0483,1,14,1,-33.514420,0.05642,...,-4.946009,-0.021828,0.170958,-0.311259,-28.294451,0.074093,-5.582651,-0.025793,0.197306,-0.348119
3,08db4255286f,3,2018-11-05 14:00:15+00:00,-34.282101,0.0680,1,14,1,-33.514420,0.05642,...,-5.351007,-0.002128,0.184957,-0.030344,-28.294451,0.074093,-5.987650,-0.006093,0.211619,-0.082239
4,08db4255286f,4,2018-11-05 14:00:20+00:00,-34.385799,0.0768,1,14,1,-33.514420,0.05642,...,-5.454706,0.006672,0.188541,0.095140,-28.294451,0.074093,-6.091349,0.002707,0.215284,0.036531
5,08db4255286f,5,2018-11-05 14:00:25+00:00,-34.925598,0.0511,1,14,1,-34.330479,0.05770,...,-5.994505,-0.019028,0.207199,-0.271332,-28.294451,0.074093,-6.631147,-0.022993,0.234362,-0.310329
6,08db4255286f,6,2018-11-05 14:00:30+00:00,-30.513399,0.1073,1,14,1,-33.596802,0.07030,...,-1.582306,0.037172,0.054692,0.530059,-28.294451,0.074093,-2.218948,0.033207,0.078423,0.448173
7,08db4255286f,7,2018-11-05 14:00:35+00:00,-30.509399,0.0649,1,14,1,-32.923260,0.07362,...,-1.578306,-0.005228,0.054554,-0.074549,-28.294451,0.074093,-2.214949,-0.009193,0.078282,-0.124078
8,08db4255286f,8,2018-11-05 14:00:40+00:00,-32.880600,0.0485,1,14,1,-32.642960,0.06972,...,-3.949507,-0.021628,0.136514,-0.308407,-28.294451,0.074093,-4.586149,-0.025593,0.162087,-0.345420
9,08db4255286f,9,2018-11-05 14:00:45+00:00,-34.674999,0.0462,1,14,1,-32.700798,0.06360,...,-5.743906,-0.023928,0.198537,-0.341205,-28.294451,0.074093,-6.380548,-0.027893,0.225505,-0.376462


In [22]:
test=feature_programming(test,delta_t=[5,10,15,20,25,30,40,50])

# model select

In [23]:
train.columns

Index(['series_id', 'step', 'timestamp', 'anglez', 'enmo', 'awake', 'hour',
       'week_name', 'anglez_roll_5', 'enmo_roll_5', 'anglez_momentum_5',
       'enmo_momentum_5', 'ratio_ang_5', 'ratio_enm_5', 'anglez_roll_10',
       'enmo_roll_10', 'anglez_momentum_10', 'enmo_momentum_10',
       'ratio_ang_10', 'ratio_enm_10', 'anglez_roll_15', 'enmo_roll_15',
       'anglez_momentum_15', 'enmo_momentum_15', 'ratio_ang_15',
       'ratio_enm_15', 'anglez_roll_20', 'enmo_roll_20', 'anglez_momentum_20',
       'enmo_momentum_20', 'ratio_ang_20', 'ratio_enm_20', 'anglez_roll_25',
       'enmo_roll_25', 'anglez_momentum_25', 'enmo_momentum_25',
       'ratio_ang_25', 'ratio_enm_25', 'anglez_roll_30', 'enmo_roll_30',
       'anglez_momentum_30', 'enmo_momentum_30', 'ratio_ang_30',
       'ratio_enm_30', 'anglez_roll_40', 'enmo_roll_40', 'anglez_momentum_40',
       'enmo_momentum_40', 'ratio_ang_40', 'ratio_enm_40', 'anglez_roll_50',
       'enmo_roll_50', 'anglez_momentum_50', 'enmo_momentum

In [24]:
X=train[['anglez', 'enmo', 'awake', 'hour',
       'week_name', 'anglez_roll_5', 'enmo_roll_5',
       'anglez_momentum_5', 'enmo_momentum_5', 'ratio_ang_5', 'ratio_enm_5',
       'anglez_roll_10', 'enmo_roll_10', 'anglez_momentum_10',
       'enmo_momentum_10', 'ratio_ang_10', 'ratio_enm_10', 'anglez_roll_15',
       'enmo_roll_15', 'anglez_momentum_15', 'enmo_momentum_15',
       'ratio_ang_15', 'ratio_enm_15', 'anglez_roll_20', 'enmo_roll_20',
       'anglez_momentum_20', 'enmo_momentum_20', 'ratio_ang_20',
       'ratio_enm_20', 'anglez_roll_25', 'enmo_roll_25', 'anglez_momentum_25',
       'enmo_momentum_25', 'ratio_ang_25', 'ratio_enm_25', 'anglez_roll_30',
       'enmo_roll_30', 'anglez_momentum_30', 'enmo_momentum_30',
       'ratio_ang_30', 'ratio_enm_30', 'anglez_roll_40', 'enmo_roll_40',
       'anglez_momentum_40', 'enmo_momentum_40', 'ratio_ang_40',
       'ratio_enm_40', 'anglez_roll_50', 'enmo_roll_50', 'anglez_momentum_50',
       'enmo_momentum_50', 'ratio_ang_50', 'ratio_enm_50', 'anglez_roll_60',
       'enmo_roll_60', 'anglez_momentum_60', 'enmo_momentum_60',
       'ratio_ang_60', 'ratio_enm_60']]
y=train['awake']

In [26]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV


import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize']=12,4


#XGBoost models and cv
def modelfit(alg,x,y,useTrainCV=True,cv_folds=5,early_stopping_rounds=50):
  #params:alg
  if useTrainCV:
    xgb_param=alg.get_xgb_params()
    xgtrain=xgb.DMatrix(x.values,label=y.values)
    cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
        metrics='auc', early_stopping_rounds=early_stopping_rounds)
    alg.set_params(n_estimators=cvresult.shape[0])
 
    #Fit the algorithm on the data
  alg.fit(x,y,eval_metric='auc')
 
#Predict training set:
  dtrain_predictions = alg.predict(x)
  dtrain_predprob = alg.predict_proba(x)[:,1]
 
#Print model report:
  print ("\nModel Report")
  print ("Accuracy : %.4g" % metrics.accuracy_score(y.values, dtrain_predictions))
  print ("AUC Score (Train): %f" % metrics.roc_auc_score(y, dtrain_predprob))
 
  feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
  feat_imp.plot(kind='bar', title='Feature Importances')
  plt.ylabel('Feature Importance Score')

In [ ]:
#learning rate and tree_based
#initialization

xgb1=XGBClassifier(learning_rate=0.1,
                   n_estimators=1000,
                   max_depth=5,
                   min_child_weight=1,
                   gamma=0,
                   subsample=0.8,
                   colsample_bytree=0.8,
                   objective='binary:logistic',
                   nthread=4,
                   scale_pos_weight=1,
                   seed=27)
modelfit(xgb1,X,y)